In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import sys
MY_DRIVE_PATH = '/content/drive/MyDrive/Technion/Diffusion-AD-Project/'
sys.path.append(MY_DRIVE_PATH)
os.chdir(MY_DRIVE_PATH)

Mounted at /content/drive


In [2]:
!pip install blobfile
!pip install mpi4py
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pytorch_lightning as pl
from IPython.display import Markdown as md
from utils.models import ModelLoader
from utils.noiser import *
from utils.denoiser import *
from utils.error_map import *
from utils.anomaly_scorer import *
from diffusion_ad.diffusion_ad import DiffusionAD
from extern.guided_diffusion.guided_diffusion.unet import UNetModel
from extern.guided_diffusion.guided_diffusion import gaussian_diffusion as gd
from config.configuration import DIFFUSION_AD_HPARAMS, DIFFUSION_AD_REQUIRED_HPARAMS, CATEGORY_TO_NOISE_TIMESTEPS

In [4]:
model: UNetModel = None

# **<u>Initialize Experiment</u>**

First, we load and construct all the needed objects for our experiment.

In [5]:
if not model:
    model, diffusion = ModelLoader().get_model('256x256_uncond')

In [6]:
noiser = TimestepUniformNoiser(diffusion)
denoiser = ModelTimestepUniformDenoiser(model, diffusion)
anomaly_map_generator = BatchFilteredSquaredError()
anomaly_scorer = MaxValueAnomalyScorer()

In [7]:
diffusion_ad = DiffusionAD(noiser,
                           denoiser,
                           anomaly_map_generator,
                           anomaly_scorer,
                           DIFFUSION_AD_HPARAMS)

In [8]:
trainer = {
    category: pl.Trainer.from_argparse_args(diffusion_ad.args, 
                                        default_root_dir=os.path.join(diffusion_ad.args.root_output_dir, category), 
                                        max_epochs=diffusion_ad.args.num_epochs,
                                        accelerator='gpu',
                                        devices=1) \
           for category in CATEGORY_TO_NOISE_TIMESTEPS.keys()
}

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utili

# **<u>Run Experiment</u>**

## **<u>Object Categories</u>**

First, we will run the evaluation on the different object categories.

In [9]:
OBJECT_CATEGORIES = [
    'hazelnut'
]

md(f'**<u>The object categories are:</u>**\n\n{OBJECT_CATEGORIES}')

**<u>The object categories are:</u>**

['hazelnut']

In [10]:
for category in OBJECT_CATEGORIES:
    md(f'**<u>Evaluating {category}</u>**')
    diffusion_ad.args.category = category
    trainer[category].test(diffusion_ad)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Total image-level auc-roc score :
1.0
test_epoch_end
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         img_auc                    1.0
        pixel_auc                  -1.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
